本节将介绍在训练过程中如何更好地访问并控制模型内部过程的方法。使用 `model.fit()`或 `model.fit_generator()` 在一个大型数据集上启动数十轮的训练，有点类似于扔一架纸飞机，一开始给它一点推力，之后你便再也无法控制其飞行轨迹或着陆点。如果想要避免不好的结果（并避免浪费纸飞机），更聪明的做法是不用纸飞机，而是用一架无人机，它可以感知其环境，将数据发回给操纵者，并且能够基于当前状态自主航行。

我们下面要介绍的技术，可以**让`model.fit()`的调用从纸飞机变为智能的自主无人机，可以自我反省并动态地采取行动**。

# TensorBoard 简介： TensorFlow 的可视化框架

想要做好研究或开发出好的模型，在实验过程中你需要丰富频繁的反馈，从而知道模型内部正在发生什么。这正是运行实验的目的：获取关于模型表现好坏的信息，越多越好。取得进展是一个反复迭代的过程（或循环）：首先你有一个想法，并将其表述为一个实验，用于验证你的想法是否正确。你运行这个实验，并处理其生成的信息。这又激发了你的下一个想法。在这个循环中实验的迭代次数越多，你的想法也就变得越来越精确、越来越强大。 Keras 可以帮你在最短的时间内将想法转化成实验，而高速 GPU 可以帮你尽快得到实验结果。但如何处理实验结果呢？这就需要 TensorBoard 发挥作用了。

![image.png](../img/07/Tensorboard.png)

本节将介绍 TensorBoard，一个内置于 TensorFlow 中的基于浏览器的可视化工具。注意，只有当 Keras 使用 TensorFlow 后端时，这一方法才能用于 Keras 模型。TensorBoard 的主要用途是，在训练过程中帮助你以可视化的方法监控模型内部发生的一切。如果你监控了除模型最终损失之外的更多信息，那么可以更清楚地了解模型做了什么、没做什么，并且能够更快地取得进展。 TensorBoard 具有下列巧妙的功能，都在浏览器中实现。
* 在训练过程中以可视化的方式监控指标
* 将模型架构可视化
* 将激活和梯度的直方图可视化
* 以三维的形式研究嵌入

In [4]:
# 在 IMDB 情感分析任务上训练一个一维卷积神经网络
# 只考虑 IMDB 词表中的前 2000 个单词，这样更易于将词嵌入可视化

# 使用了 TensorBoard 的文本分类模型
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,input_length=max_len,name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 在开始使用 TensorBoard 之前，我们需要创建一个目录，用于保存它生成的日志文件

# 为 TensorBoard 日志文件创建一个目录

!mkdir my_log_dir

子目录或文件 my_log_dir 已经存在。


In [6]:
# 使用一个 TensorBoard 回调函数来训练模型

# 回调函数会将日志事件写入硬盘的指定位置

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',      # 日志文件将被写入这个位置
        histogram_freq=1,          # 每一轮之后记录激活直方图
        embeddings_freq=1,         # 每一轮之后记录嵌入数据
        embeddings_data = np.arange(0, max_len).reshape((1, max_len)),
    )
]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 8s 387us/step - loss: 0.6446 - acc: 0.6503 - val_loss: 0.9186 - val_acc: 0.6822


InvalidArgumentError: You must feed a value for placeholder tensor 'embed_input' with dtype float and shape [?,500]
	 [[Node: embed_input = Placeholder[dtype=DT_FLOAT, shape=[?,500], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: embed/embeddings/read/_169 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_18_embed/embeddings/read", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'embed_input', defined at:
  File "c:\python\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python\lib\site-packages\traitlets\config\application.py", line 658, in lau

embeddings_data: data to be embedded at layers specified in embeddings_layer_names. Numpy array (if the model has a single input) or list of Numpy arrays (if the model has multiple inputs).

在命令行启动 TensorBoard 服务器，指示它读取回调函数当前正在写入的日志。然后可以用浏览器打开 http://localhost:6006 ，并查看模型的训练过程。除了训练指标和验证指标的实时图表之外，你还可以访问 HISTOGRAMS（直方图）标签页，并查看美观的直方图可视化，直方图中是每层的激活值

In [ ]:
!tensorboard --logdir=my_log_dir

Keras 还提供了另一种更简洁的方法——`keras.utils.vis_utils.plot_model` 函数，它可以将模型绘制为层组成的图，而不是 TensorFlow 运算组成的图。使用这个函数需要安装 Python 的pydot 库和 pydot-ng 库，还需要安装 graphviz 库。

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png',show_layer_names=True)